In [1]:
import pandas as pd
import sqlite3 as db
from langdetect import detect
import chess
import chess.pgn
import io
import utils

In [2]:
con = db.connect("./chess.db")
games = pd.read_sql_query("SELECT * FROM gameknot_games", con)
games

,id,pgn,language
0,44,"[Event ""alygally&#039;s mini-tournament""]\n[Da...",en
1,62,"[Event ""League division D2""]\n[Date ""08-Jun-06...",en
2,64,"[Event ""Let&#039;s play chess""]\n[Date ""25-May...",en
3,66,"[Event ""easy street""]\n[Date ""26-Jun-06""]\n[Wh...",en
4,70,"[Event ""001 mini-tournament""]\n[Date ""31-May-0...",en
...,...,...,...
12736,70354,"[Event ""A question of queens...""]\n[Date ""14 S...",en
12737,70355,"[Event ""50th World Open U1800 Round 9 '22""]\n[...",en
12738,70359,"[Event ""Team match""]\n[Date ""2022.06.26""]\n[Wh...",en
12739,70364,"[Event ""Team match""]\n[Date ""2022.06.30""]\n[Wh...",en


In [3]:
utils.games_to_moves(games.head(3))

,position,move,comment,halfmove_number,game_id
0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,None,This is my first gameknot game against someone...,1,44
1,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,e7e5,"I've been playing the Vienna Gambit as white, ...",3,44
2,rnbqkbnr/pppp1ppp/8/4p3/4P3/2N5/PPPP1PPP/R1BQK...,b1c3,Minor disappointment.,4,44
3,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2NP4/PPP2PP...,b8c6,"My idea here is to expand on the kingside, dri...",9,44
4,r1bqk2r/ppp2ppp/3p1n2/n1b1pP2/2B1P3/2NP4/PPP3P...,c6a5,"Maybe this isn't the greatest plan, since with...",13,44
5,r1bqk2r/pp3ppp/2pp1n2/2b1pP2/2P1P3/2N2Q2/PPP3P...,c7c6,Going for the kingside attack anyway!,17,44
6,r1bqk2r/pp3p1p/2pp2p1/2b1pPPn/2P1P3/2N2Q2/PPP4...,f6h5,"Now everything is still defended adequately, a...",21,44
7,r1bqk2r/pp3p1p/2pp2p1/2b1pPPn/2P1P3/2N2Q2/PPP1...,g1e2,"? Too slow, maybe.",22,44
8,r1bqk2r/5p1p/p1pp2p1/1pb1pPP1/2P1P3/2N2QP1/PPP...,b7b5,"Why do this? White has doubled pawns already,...",27,44
9,r1bqk2r/5p1p/p1pp2p1/2b1pPP1/2P1P3/2N2QP1/P1P5...,b3c4,"Pins the knight, which is defended by the quee...",30,44


In [4]:
game = chess.pgn.read_game(io.StringIO(games.pgn[0]))

In [5]:
for node in game.mainline():
    print(node.parent.board())
    print(node.move, node.ply())
    print("-----------")

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
e2e4 1
-----------
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
e7e5 2
-----------
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
b1c3 3
-----------
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R
f8c5 4
-----------
r n b q k . n r
p p p p . p p p
. . . . . . . .
. . b . p . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R
f1c4 5
-----------
r n b q k . n r
p p p p . p p p
. . . . . . . .
. . b . p . . .
. . B . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K . N R
g8f6 6
-----------
r n b q k . . r
p p p p . p p p
. . . . . n . .
. . b . p . . .
. . B . P . . .
. . N . . . . .
P P P P . P P P
R . B 